In [29]:
%pip install python-dotenv langchain langchain-text-splitters langchain_pinecone
%pip install -qU pymupdf
%pip install --upgrade pip


  Using cached langchain_pinecone-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached pinecone_client-5.0.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain_pinecone-0.2.0-py3-none-any.whl (11 kB)
Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl (392 kB)
Using cached pinecone_client-5.0.1-py3-none-any.whl (244 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to us

In [30]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
loader = PyMuPDFLoader('./year-end-tax-settlement-guide-2024.pdf')
docs = loader.load()

document_list = text_splitter.split_documents(docs)

/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 15 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(
/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 37 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(
/Users/a13517/.pyenv/versions/3.12.7/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 223 of document ./year-end-tax-settlement-guide-2024.pdf
  warnings.warn(


In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv()

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'year-end-tax-settlement-index'
database = Pinecone.from_documents(
    documents=document_list,
    embedding=embeddings,
    index_name=index_name
)
